In [2]:
from pyspark import SparkConf, SparkContext

ModuleNotFoundError: No module named 'pyspark'

In [3]:
spark_conf = SparkConf()\
        .setAppName("Assignment 1")
sc=SparkContext.getOrCreate(spark_conf) 
data = sc.textFile("AllVideos.csv")

NameError: name 'SparkConf' is not defined

In [3]:
# 3.1 Controversial Trending Videos Identification

In [4]:
# extract data needed for the further analysized
def extractInfo(record):
    try:
        video_id,trending_date,category,views,likes,dislikes,country = record.split(",")
        return ((video_id, country),(trending_date,views,likes,dislikes))
#         return (video_id,trending_date,likes, dislikes,country)
    except:
        return ()

In [5]:
# calculate growth value
def calculate(record):
    try:
        size = len(record)
#         last data - first data
        Ldifference = int(record[size-1][2]) - int(record[0][2])
        Ddifference = int(record[size-1][3]) - int(record[0][3])
        return(Ddifference - Ldifference)
    except:
        return()

In [6]:
# extract data and display all the apperant values
ProcessData = data.map(extractInfo)\
             .groupByKey()\
             .mapValues(list)

Trend = ProcessData.sortBy(lambda r:r[1][0],ascending = True)\
                   .mapValues(calculate)\
                   .map(lambda r:(r[0][0],r[1],r[0][1]))\
                   .sortBy(lambda r:r[1],ascending = False)
Trend.take(10)

[('QwZT7T-TXT0', 579119, 'GB'),
 ('QwZT7T-TXT0', 478100, 'US'),
 ('BEePFpC9qG8', 365862, 'DE'),
 ('RmZ3DPJQo2k', 334390, 'KR'),
 ('q8v9MvManKE', 299044, 'IN'),
 ('pOHQdIDds6s', 160365, 'CA'),
 ('ZGEoqPpJQLE', 151913, 'RU'),
 ('84LBjXaeKk4', 134836, 'FR'),
 ('84LBjXaeKk4', 134834, 'DE'),
 ('84LBjXaeKk4', 121240, 'RU')]

In [7]:
# 3.2 Category and Trending Correlation

In [8]:
# extract data()
def extract(record):
    try:
        video_id,trending_date,category,views,likes,dislikes,country = record.split(",")
        return ((category, video_id), country)
    except:
        return ()

In [9]:
# get all the countries that individuals ids has apperead 
def countries(countries, country):
    if country not in countries:
        countries = countries + "|" + country

    return countries

In [10]:
# get all total amount of the countries
def get_length(lists):
    country_list = lists[1].strip().split("|")
    return (lists[0], len(set(country_list)))

In [11]:
# remove the video_id in this function return category,total amount
def remove_movie_id(lists):
    i,j = lists[0], lists[1]
    new = i[0]
    return (new, j)

In [12]:
# get the the cumulative sum of countries included in each id, all ids appeared in a certain category
def cumulativeSum(lists, current_country_number):
    total_country_number, total_movie_id_number = lists
    total_country_number  = total_country_number + current_country_number
    total_movie_id_number = total_movie_id_number + 1
    return (total_country_number,total_movie_id_number)

In [13]:
def merge_numbers(pair1, pair2):
    country_number1 ,id_number1 = pair1
    country_number2 ,id_number2 = pair2
    return (country_number1+country_number2, id_number1+id_number2)

In [14]:
def AverageCalculation(lists):
    category, total_countries_ids = lists

    average_trending = total_countries_ids[0] / total_countries_ids[1]
    return (category,average_trending)

In [15]:
# extract necessary data to perform analysis 
raw_data = data.map(extract)
# get total amount of the countries that has the same video_id showns
# data = raw_data.reduceByKey(ids_country).map(total_amount)
datas = raw_data.reduceByKey(countries).map(get_length)
# aggregateBykey(initialvalue,
# operation when key is same 
# operation when key is different )
Average = datas.map(remove_movie_id)\
          .aggregateByKey((0.0, 0.0), cumulativeSum, merge_numbers, 1)\
#           .map(AverageCalculation)
# list out all the content
Average.take(200)

[('Entertainment', (60617.0, 52959.0)),
 ('People & Blogs', (35405.0, 33279.0)),
 ('News & Politics', (23367.0, 22197.0)),
 ('Sports', (15475.0, 13549.0)),
 ('Film & Animation', (10784.0, 9235.0)),
 ('Music', (13805.0, 10534.0)),
 ('Comedy', (13042.0, 10741.0)),
 ('Nonprofits & Activism', (2102.0, 1988.0)),
 ('Howto & Style', (10599.0, 9746.0)),
 ('Education', (4343.0, 4086.0)),
 ('Science & Technology', (4038.0, 3473.0)),
 ('Pets & Animals', (2495.0, 2331.0)),
 ('Gaming', (6119.0, 5591.0)),
 ('Travel & Events', (929.0, 850.0)),
 ('Autos & Vehicles', (3478.0, 3413.0)),
 ('Shows', (678.0, 436.0)),
 ('Movies', (15.0, 12.0)),
 ('Trailers', (3.0, 3.0))]